In [1]:
import patient as p
import tensorflow as tf
import numpy as np
import myLib
import consts as c



In [2]:
c.WINDOW_SIZE = c.PREPREDICTION_LENGTH + 1000

tabSegments = []
for pStr in c.trainPatients:
    tabSegments = tabSegments + p.patient(pStr).getLabeledSegments()


x_train,y_train = myLib.processDF(tabSegments)
x_validation,y_validation = myLib.processDF(p.patient(c.validationPatient).getLabeledSegments())
x_test,y_test = myLib.processDF(p.patient(c.testPatient).getLabeledSegments())




Loading data of patient MSEL_00172


100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [00:12<00:00, 20.04it/s]


Loading data of patient MSEL_00501


100%|████████████████████████████████████████████████████████████████████████████████| 277/277 [00:13<00:00, 19.98it/s]


Loading data of patient MSEL_01097


...

100%|████████████████████████████████████████████████████████████████████████████████| 325/325 [00:16<00:00, 20.26it/s]


Loading data of patient MSEL_01838


100%|████████████████████████████████████████████████████████████████████████████████| 404/404 [00:20<00:00, 19.39it/s]


Loading data of patient MSEL_01808


100%|████████████████████████████████████████████████████████████████████████████████| 325/325 [00:17<00:00, 19.07it/s]


In [3]:
print(x_train[10])

[-0.31760122 -0.39565403 -1.03526466 ... -0.39547062 -1.02448492
 -1.33243008]


In [4]:
print(f"train data: {len(x_train)}\t validation: {len(x_validation)}\t test:{len(x_test)}\n")
unique, counts = np.unique(y_train, return_counts=True)

print(f"TRAIN:     Negatives: {counts[0]} Positives: {counts[1]}")
pos = counts[1]
neg = counts[0]
ratio = round(100 / len(x_train) * pos)
unique, counts = np.unique(y_validation, return_counts=True)
print(f"VALIDATION Negatives: {counts[0]} Positves: {counts[1]}")
unique, counts = np.unique(y_test, return_counts=True)
print(f"TEST       Negatives: {counts[0]} Positves: {counts[1]}")


print(f"\n{ratio} % of positives")




train data: 2142	 validation: 397	 test:331

TRAIN:     Negatives: 2008 Positives: 134
VALIDATION Negatives: 386 Positves: 11
TEST       Negatives: 322 Positves: 9

6 % of positives


In [5]:

def getModel1():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(2,activation=tf.nn.softmax))

    #model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
          loss=tf.keras.losses.BinaryCrossentropy(),
          metrics=[
              tf.keras.metrics.BinaryAccuracy(name='accuracy')
          ]
    )
    return model

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def getModel2(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(
          16, activation='relu',
          input_shape=(x_train.shape[-1],)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model



In [6]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

initial_bias = np.log([pos/neg])

model = getModel2(output_bias=initial_bias)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                82000     
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 82,017
Trainable params: 82,017
Non-trainable params: 0
_________________________________________________________________


In [7]:
import math
results = model.evaluate(x_train, y_train, batch_size=BATCH_SIZE, verbose=0)
p0 = pos/(pos+neg)
expectedInitLoss = -p0 * math.log(p0) - (1 - p0) * math.log(1-p0)
print(f"Intial Loss: {round(results[0],4)} expected : {round(expectedInitLoss,4)}")

Intial Loss: 0.2964 expected : 0.2339


In [8]:
import os
import tempfile
initial_weights = os.path.join(tempfile.mkdtemp(),"initial_weights")
model.save_weights(initial_weights)

In [9]:
model = getModel2()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(x_validation, y_validation), 
    verbose=0)

In [ ]:
model = getModel2()
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(x_validation, y_validation), 
    verbose=0)

In [ ]:
import matplotlib.pyplot as plt


def plot_loss(history, label, color = "C1"):
    # Use a log scale on y-axis to show the wide range of values.
    plt.semilogy(history.epoch, history.history['loss'],
               color=color, label='Train ' + label)
    plt.semilogy(history.epoch, history.history['val_loss'],
               color=color, label='Val ' + label,
               linestyle="--")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')


plot_loss(zero_bias_history, "Zero Bias","C0")
plot_loss(careful_bias_history, "Careful Bias","C2")

In [ ]:
model = getModel2()
model.load_weights(initial_weights)
baseline_history = model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(x_validation, y_validation), 
    verbose=0)

In [ ]:
def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color="C1", label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color="C0", linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

plot_metrics(baseline_history)

In [ ]:
val_loss = model.evaluate(x_test,y_test)
print(f"\nloss: {val_loss}")

predictions = model.predict(x_test)
res = [np.argmax(x) for x in predictions]
for i in range(len(y_test)):
    print(f"{y_test[i]} => {res[i]}")